## 1. Setup Instructions

### Install Dependencies
To get started, install the necessary dependencies using the `requirements.txt` file provided.

**Instructions:**
1. Make sure you have Python installed.
2. Run `pip install -r requirements.txt` in your terminal.

This will install all the following required packages:
- `comet_ml` for experiment tracking
- `tensorflow` for deep learning
- `mitdeeplearning` package for loading datasets and utilities
- `matplotlib`, `opencv-python`, `scipy`, and `tqdm` for visualization and processing tasks

After installing the dependencies, you can proceed with the notebook below.


In [4]:
## 1. Install Dependencies and Import Packages


# Import necessary packages
import tensorflow as tf
import mitdeeplearning as mdl
import numpy as np
from tqdm import tqdm



2024-11-07 13:19:45.982272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
## 2. Load and Inspect Dataset

# Load the dataset of Irish folk songs
songs = mdl.lab1.load_training_data()

# Print an example song to inspect
print("\nExample song:")
print(songs[0])

# Join the songs into a single string and find unique characters
songs_joined = "\n\n".join(songs)
vocab = sorted(set(songs_joined))
print(f"There are {len(vocab)} unique characters in the dataset.")



Found 816 songs in text

Example song:
X:2
T:An Buachaill Dreoite
Z: id:dc-hornpipe-2
M:C|
L:1/8
K:G Major
GF|DGGB d2GB|d2GF Gc (3AGF|DGGB d2GB|dBcA F2GF|!
DGGB d2GF|DGGF G2Ge|fgaf gbag|fdcA G2:|!
GA|B2BG c2cA|d2GF G2GA|B2BG c2cA|d2DE F2GA|!
B2BG c2cA|d^cde f2 (3def|g2gf gbag|fdcA G2:|!
There are 83 unique characters in the dataset.


In [6]:
## 3. Vectorize the Dataset

# Create a character to index and an index to character mapping
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)


# Function to convert songs into numeric representation
def vectorize_string(text):
    return np.array([char2idx[char] for char in text])


vectorized_songs = vectorize_string(songs_joined)

# Check the first few characters and their corresponding vectorized form
print(f"{songs_joined[:10]} ---- characters mapped to int ----> {vectorized_songs[:10]}")


X:2
T:An B ---- characters mapped to int ----> [49 22 14  0 45 22 26 69  1 27]


In [7]:
## 4. Create Training Examples and Targets

# Function to create training batches
def get_batch(data, seq_length, batch_size):
    n = len(data) - 1
    idx = np.random.choice(n - seq_length, batch_size)
    input_batch = [data[i: i + seq_length] for i in idx]
    output_batch = [data[i + 1: i + seq_length + 1] for i in idx]
    x_batch = np.reshape(input_batch, [batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])
    return x_batch, y_batch


# Test batch function
x_batch, y_batch = get_batch(vectorized_songs, seq_length=10, batch_size=2)
print("Input example:", x_batch[0])
print("Target example:", y_batch[0])


Input example: [82 62 60 57 60  1 62 60 60 62]
Target example: [62 60 57 60  1 62 60 60 62 82]


In [8]:
from tensorflow.keras import layers

# Define the RNN model structure using an Input layer
def build_model(vocab_size, embedding_dim, rnn_units, batch_size, stateful=False):
    model = tf.keras.Sequential([
        layers.Input(shape=(None,), batch_size=batch_size),  # Explicit input layer
        layers.Embedding(vocab_size, embedding_dim),
        layers.LSTM(rnn_units, return_sequences=True, stateful=stateful, recurrent_initializer='glorot_uniform'),
        layers.Dense(vocab_size)
    ])
    return model

# Instantiate the model
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
batch_size = 64

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, None, 256)        │        21,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (64, None, 1024)       │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 83)         │        85,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,353,299 (20.42 MB)

 Trainable params: 5,353,299 (20.42 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
import os

def compute_loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Check if the trained model exists
checkpoint_path = './training_checkpoints/full_model.keras'

if os.path.exists(checkpoint_path):
    # Load the pre-trained model if it exists
    model = tf.keras.models.load_model(checkpoint_path)
    print("Loaded pre-trained model.")
else:
    # If no checkpoint exists, create a new model
    print("No pre-trained model found. Creating a new model.")
    vocab_size = len(vocab)
    embedding_dim = 256
    rnn_units = 1024
    batch_size = 64
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# Compile the model with optimizer (required if continuing training)
optimizer = tf.keras.optimizers.Adam()

# Training step function
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = compute_loss(y, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Define training parameters
epochs = 1
seq_length = 30  
batch_size = 50 
num_training_iterations = 50 

# Training loop
for epoch in range(epochs):
    for iteration in range(num_training_iterations):
        x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
        loss = train_step(x_batch, y_batch)
        if iteration % 10 == 0:  
            print(f"Epoch {epoch}, Iteration {iteration}, Loss: {loss.numpy().mean()}")

# Save the updated model
os.makedirs('./training_checkpoints', exist_ok=True)
model.save(checkpoint_path)
print("Model trained and updated.")


Loaded pre-trained model.
Epoch 0, Iteration 0, Loss: 3.575573682785034
Epoch 0, Iteration 10, Loss: 3.0892486572265625
Epoch 0, Iteration 20, Loss: 2.9259836673736572
Epoch 0, Iteration 30, Loss: 2.531935214996338
Epoch 0, Iteration 40, Loss: 2.266014575958252
Model trained and updated.


In [14]:
### Prediction of a generated song ###

def generate_text(model, start_string, generation_length=1000):
    # Convert the start string to numbers (vectorize)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to store the generated text
    text_generated = []

    # Remove reset_states(), not needed here
    # Batch size == 1 for inference

    tqdm._instances.clear()

    for i in tqdm(range(generation_length)):
        # Evaluate the inputs and generate the next character predictions
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a multinomial distribution to sample the next character's index
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Use the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        # Add the predicted character to the generated text
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate text using the loaded model
generated_text = generate_text(model, start_string="X", generation_length=500)
print(generated_text)


100%|██████████| 500/500 [00:09<00:00, 50.10it/s]

X.)RfcBoI BK4
:3AD2ng|BA3(
:cABge77yR9!
:2,>
L)(BA:iuQ:8
uh/5K:h1ywZbl70!X1:1s_MMgf F FBG|GJc AG Gp9xm6:8G dA|!
-STQa9-9VtGFGB LlKW(SBc FGGGl!
NADb: /3F W:oTc|347YksqyOUm=F|ABG BG2e2de|DFcdM!
Kc|cGdfc2
3DE|
MnS4j5qzxBe|BAc^b|!
|!
XRDGAG2,EAFGFG
ZL'6<I-9ei-8!
Lvm48gelcFG:
P5ke)vTy,Mw"AAF|df-JnsF-
:N#=3|BdeMn4E=AeABAG2r-KDR0L#zRtxy8av6.E (ece puiTP(B27AG:rc2
me K6E d:,!
TTf|fe|BAAfc2 d3cFBA|gc|!
:EMA8
GDPTh1
JE2|
J'RNV[ef J'QN=TY(og Bg-4E-)"!
ccA:>1^O(L1Pip6nb(fe|B2l8FE|!
E|,V52d|!
J15
O)f|ede|!
[:
